# مشروع تنقيب البيانات – الهواتف الذكية


In [ ]:
# -*- coding: utf-8 -*-

# 1️⃣ استيراد المكتبات الأساسية

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
#################
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
########################

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [ ]:
# 2️⃣ تحميل البيانات
df = pd.read_csv("smartphones.csv")
# عرض أول 5 صفوف
df.head() # ضع اسم الملف هنا

In [ ]:
df_copy = df.copy()

# حذف اللون لانه غير مهم في السعر غالبا

In [ ]:
df.drop(columns=['Color'], inplace=True, errors='ignore')


df.head()

In [ ]:
# تحويل Yes إلى 1 و No إلى 0
df['Free'] = df['Free'].map({'Yes': 1, 'No': 0})

# التأكد من التحويل
print(df.head())


In [ ]:
# حساب تكرار كل ماركة
brand_counts = df['Brand'].value_counts()

# اختيار الماركات اللي عددها >= 20
brands_to_keep = brand_counts[brand_counts >= 20].index

# الاحتفاظ بالصفوف اللي ماركتها ضمن brands_to_keep
df = df[df['Brand'].isin(brands_to_keep)]

# التأكد
print(df['Brand'].value_counts())


In [ ]:
# أولاً: تحديد الماركات اللي نحتفظ فيها (كل الماركات اللي عددها >= 20)
brands_to_keep = ['Samsung','Xiaomi','Apple','Realme','OPPO','POCO',
                  'Motorola','Huawei','TCL','Cubot','Ulefone','Vivo',
                  'Blackview','Honor','OnePlus','Hammer']

# الاحتفاظ بالصفوف اللي ماركتها ضمن brands_to_keep
df = df[df['Brand'].isin(brands_to_keep)]

# تحويل العمود Brand لأرقام
le = LabelEncoder()
df['Brand'] = le.fit_transform(df['Brand'])

# التأكد من التحويل
print(df[['Brand']].head())
print(df['Brand'].value_counts())


In [ ]:
# أعلى قيمة
max_price = df['Final Price'].max()

# أقل قيمة
min_price = df['Final Price'].min()

print("أعلى قيمة:", max_price)
print("أقل قيمة:", min_price)

# الاحتفاظ فقط بالقيم الطبيعية للهواتف
df = df[(df['Final Price'] >= 50) & (df['Final Price'] <= 2000)]


# عرض القيم
df['Final Price']

# نقوم بمعالجة القم الفارغة ولبمتكرره

In [ ]:
print(df.isnull().sum()) 

In [ ]:
# حذف الصفوف التي تحتوي على أي قيمة فارغة
df = df.dropna()

# التأكد
print(df.isnull().sum())  # يجب أن يكون كل الأعمدة صفر
print(df.shape)           # يعرض عدد الصفوف والأعمدة بعد الحذف


In [ ]:
# # ---------------------------
# # تنظيف عمود RAM
# # ---------------------------
# allowed_ram = [2, 4, 8, 16, 32]
# most_common_ram = df.loc[df['RAM'].isin(allowed_ram), 'RAM'].mode()[0]

# df['RAM'] = df['RAM'].apply(lambda x: x if x in allowed_ram else most_common_ram)

# # ---------------------------
# # تنظيف عمود Storage
# # ---------------------------
# allowed_storage = [16, 32, 64, 128, 256, 512]
# most_common_storage = df.loc[df['Storage'].isin(allowed_storage), 'Storage'].mode()[0]

# df['Storage'] = df['Storage'].apply(lambda x: x if x in allowed_storage else most_common_storage)

# # ---------------------------
# # عرض النتائج
# # ---------------------------
# print(df[['RAM', 'Storage']].head())


In [ ]:
# تحويل عمود Model لأرقام
le_model = LabelEncoder()
df['Model'] = le_model.fit_transform(df['Model'])


for col in ['Smartphone']:
    df[col] = df[col].str.strip()

from sklearn.preprocessing import LabelEncoder

for col in ['Smartphone']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])


In [ ]:

# -------------------------------
# افترض أن df جاهز بعد تنظيف البيانات
# -------------------------------


# الميزات (Features) والهدف (Target)
X = df.drop(columns=['Final Price'])
y = df['Final Price']

# تقسيم البيانات للتدريب والاختبار
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# إنشاء نموذج Random Forest
rf = RandomForestRegressor(n_estimators=300, random_state=42)

# تدريب النموذج
rf.fit(X_train, y_train)

# التنبؤ على مجموعة الاختبار
y_pred = rf.predict(X_test)

# تقييم النموذج
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")

# حساب الدقة بالنسبة المئوية
mape = np.mean(np.abs((y_test - y_pred) / y_test))
accuracy = 1 - mape
print(f"الدقة: {accuracy*100:.2f}%")


In [ ]:
importances = rf.feature_importances_
features = X.columns

for feature, importance in zip(features, importances):
    print(f"{feature}: {importance:.2f}")

# هذه الخطوة تبين لك أي الأعمدة أكثر تأثير على سعر الهاتف.

In [ ]:
# الجزء 10: حفظ نسخة بعد المعالجة
df.to_csv("smartphonesحفظ الملف بعد التعديل.csv", index=False)
print("تم حفظ نسخة البيانات بعد المعالجة باسم: smartphones_processed.csv")